In [ ]:
import pandas as pd

# Loading Datasets

In [ ]:
#Loads files from github
covid_url ='https://raw.githubusercontent.com/zhrmnch/CSI4142_Phase3/main/COVID19_cases.csv'
MCI_url ='https://raw.githubusercontent.com/zhrmnch/CSI4142_Phase3/main/Major_Crime_Indicators.csv'
neighbourhood_crime_rate_url ='https://raw.githubusercontent.com/zhrmnch/CSI4142_Phase3/main/Neighbourhood_Crime_Rates.csv'

# Prepare Crime fact table

In [ ]:
# MC Dataset
crimedf = pd.read_csv(MCI_url)
crimedf = crimedf.drop_duplicates()
crimedf['occurrencedate'] = pd.to_datetime(crimedf['occurrencedate'])
crimedf['reporteddate'] = pd.to_datetime(crimedf['reporteddate'])
crimedf = crimedf.drop(columns=["event_unique_id","Index_","X","Y","ObjectId","Hood_ID","reportedyear","reportedmonth", "reportedday", "reporteddayofyear", "reporteddayofweek", "reportedhour","occurrenceyear", "occurrencemonth", "occurrenceday", "occurrencedayofyear", "occurrencedayofweek", "occurrencehour"])

# Manual changes to the neighbourhood column to account for typos
crimedf['Neighbourhood'] = crimedf['Neighbourhood'].replace('LAmoreaux', "L'Amoreaux")
crimedf['Neighbourhood'] = crimedf['Neighbourhood'].replace('OConnor-Parkview', "O'Connor-Parkview")
crimedf['Neighbourhood'] = crimedf['Neighbourhood'].replace('Mimico', "Mimico (includes Humber Bay Shores)")
crimedf['Neighbourhood'] = crimedf['Neighbourhood'].replace('Tam OShanter-Sullivan', "Tam O'Shanter-Sullivan")

# Remove crimes with no location
crimedf = crimedf[(crimedf["Neighbourhood"] != "NSA") & (crimedf["Latitude"] != 0) & (crimedf["Longitude"] != 0)]
crimedf = crimedf.drop_duplicates()

#crimedf.describe(include = ['object','int'])


# Prepare COVID Dataset

In [ ]:
# COVID Dataset
covdf = pd.read_csv(covid_url)
covdf['Reported Date'] = pd.to_datetime(covdf['Reported Date'])
covdf['Episode Date'] = pd.to_datetime(covdf['Episode Date'])

In [ ]:
covdf.drop_duplicates()

# Prepare Neighbourhood Table

In [ ]:
# Neighbourhood Table - DONE
neighbourhooddf = pd.DataFrame()
neighbourhooddf = pd.read_csv(neighbourhood_crime_rate_url)
neighbourhooddf = neighbourhooddf.loc[:, ['Neighbourhood','Shape__Area','Shape__Length']]
neighbourhooddf = neighbourhooddf.drop_duplicates() # remove dups
neighbourhooddf['neighbourhood_PK'] = range(1, len(neighbourhooddf) + 1) # generate surr key
neighbourhooddf = neighbourhooddf[['neighbourhood_PK'] + [col for col in neighbourhooddf.columns if col != 'neighbourhood_PK']]

# Manual changes to the neighbourhood column
neighbourhooddf['Neighbourhood'] = neighbourhooddf['Neighbourhood'].replace('Mimico (includes Humber Bay Shores) ', 'Mimico (includes Humber Bay Shores)')

# Prepare Location Table

In [ ]:
# Location Table - DONE
locationdf = pd.DataFrame()
locationdf = crimedf[['Division', 'Longitude', 'Latitude']].copy()
locationdf = locationdf.drop_duplicates() # remove dups
locationdf['location_PK'] = range(1, len(locationdf) + 1) # generate surr key
locationdf = locationdf[['location_PK'] + [col for col in locationdf.columns if col != 'location_PK']]

# Prepare Premise Table

In [ ]:
# Premise Table - DONE
premisedf = pd.DataFrame()
premisedf = crimedf[['location_type','premises_type']].copy()
premisedf = premisedf.drop_duplicates() # remove dups
premisedf['premise_PK'] = range(1, len(premisedf) + 1) # generate surr key
premisedf = premisedf[['premise_PK'] + [col for col in premisedf.columns if col != 'premise_PK']]

# Prepare Incident Table

In [ ]:
# Incident Table - DONE
incidentdf = pd.DataFrame()
incidentdf = crimedf[['mci_category','offence','ucr_code', 'ucr_ext']].copy()
incidentdf = incidentdf.drop_duplicates() # remove dups
incidentdf['incident_PK'] = range(1, len(incidentdf) + 1) # generate surr key
incidentdf = incidentdf[['incident_PK'] + [col for col in incidentdf.columns if col != 'incident_PK']]

# Prepare Occuence Table

In [ ]:
# Occurrence Date Table - DONE
occurencedf = pd.DataFrame()
occurencedf = crimedf[['occurrencedate']].copy()
occurencedf = occurencedf.drop_duplicates() # remove dups
occurencedf['occurence_PK'] = range(1, len(occurencedf) + 1) # generate surr key
occurencedf = occurencedf[['occurence_PK'] + [col for col in occurencedf.columns if col != 'occurence_PK']]

# Generating new columns
occurencedf['occurrencehour'] = pd.DatetimeIndex(occurencedf['occurrencedate']).hour
occurencedf['occurrenceday'] = pd.DatetimeIndex(occurencedf['occurrencedate']).day
occurencedf['occurrencemonth'] = pd.DatetimeIndex(occurencedf['occurrencedate']).month
occurencedf['occurrenceyear'] = pd.DatetimeIndex(occurencedf['occurrencedate']).year

# Prepare Reported Table

In [ ]:
# Reported Date Table - DONE
reporteddf = pd.DataFrame()
reporteddf = crimedf[['reporteddate']].copy() # create location dimension
reporteddf = reporteddf.drop_duplicates() # remove dups
reporteddf['reported_PK'] = range(1, len(reporteddf) + 1) # generate surr key
reporteddf = reporteddf[['reported_PK'] + [col for col in reporteddf.columns if col != 'reported_PK']]

# Generating new columns
reporteddf['reportedhour'] = pd.DatetimeIndex(reporteddf['reporteddate']).hour
reporteddf['reportedday'] = pd.DatetimeIndex(reporteddf['reporteddate']).day
reporteddf['reportedmonth'] = pd.DatetimeIndex(reporteddf['reporteddate']).month
reporteddf['reportedyear'] = pd.DatetimeIndex(reporteddf['reporteddate']).year

# Stage COVID Fact Table from COVID dataframe

In [ ]:
# Monthly COVID Fact Table/Cases Table
casesdf = pd.DataFrame()
casesdf = covdf.groupby([pd.to_datetime(covdf['Episode Date']).dt.to_period('M'), 'Neighbourhood Name'])['_id'].count().reset_index()
casesdf.columns = ['Date', 'Neighbourhood', 'Cases']

# Date conversions
casesdf['date'] = casesdf['Date'].dt.to_timestamp(freq='M')
casesdf['year'] = casesdf['Date'].dt.year
casesdf['month'] = casesdf['Date'].dt.month
casesdf = casesdf.drop('Date', axis=1)

# Manual changes to typos in neighbourhood column
casesdf['Neighbourhood'] = casesdf['Neighbourhood'].replace('Cabbagetown-South St. James Town', 'Cabbagetown-South St.James Town')
casesdf['Neighbourhood'] = casesdf['Neighbourhood'].replace('Briar Hill - Belgravia', 'Briar Hill-Belgravia')
casesdf['Neighbourhood'] = casesdf['Neighbourhood'].replace('Danforth-East York', 'Danforth East York')
casesdf['Neighbourhood'] = casesdf['Neighbourhood'].replace('North St. James Town', 'North St.James Town')

# Merge COVID Fact Table with Neighbourhood Table

In [ ]:
# Generating Surr Key for COVID Month Table - DONE
casesdf = casesdf.merge(neighbourhooddf, on=['Neighbourhood'])
casesdf = casesdf.drop(['date', 'Neighbourhood', 'Shape__Area', 'Shape__Length'], axis=1)

# Merge Crime Fact Table with dimensions

In [ ]:
crimedf = crimedf.merge(premisedf, on=['location_type','premises_type'])
crimedf = crimedf.drop(['location_type','premises_type'], axis=1)
crimedf = crimedf.merge(incidentdf, on=['mci_category','offence','ucr_code', 'ucr_ext'])
crimedf = crimedf.drop(['mci_category','offence','ucr_code', 'ucr_ext'], axis=1)
crimedf = crimedf.merge(locationdf, on=['Division', 'Longitude', 'Latitude'])
crimedf = crimedf.drop(['Division', 'Longitude', 'Latitude'], axis=1)
crimedf = crimedf.merge(occurencedf, on=['occurrencedate'])
crimedf = crimedf.drop(['occurrencedate','occurrencehour','occurrenceday','occurrencemonth','occurrenceyear'], axis=1)
crimedf = crimedf.merge(reporteddf, on=['reporteddate'])
crimedf = crimedf.drop(['reporteddate','reportedhour','reportedday','reportedmonth','reportedyear'], axis=1)
crimedf = crimedf.merge(neighbourhooddf, on=['Neighbourhood'])
crimedf = crimedf.drop(['Neighbourhood','Shape__Area','Shape__Length'], axis=1)

# Save Tables as CSV

In [ ]:
# Saving tables to csv
crimedf.to_csv('CrimeFactTable.csv', index=False)
casesdf.to_csv('CovidFactTable.csv', index=False)
neighbourhooddf.to_csv('NeighbourhoodTable.csv', index=False)
incidentdf.to_csv('IncidentTable.csv', index=False)
premisedf.to_csv('PremiseTable.csv', index=False)
locationdf.to_csv('LocationTable.csv', index=False)
occurencedf.to_csv('OccurrenceTable.csv', index=False)
reporteddf.to_csv('ReportedTable.csv', index=False)